# ViaRisk — Exploratory Data Analysis (EDA)

## Contexto
Este notebook analiza los **microdatos de accidentes de tráfico con víctimas (2024)** publicados por la **Dirección General de Tráfico (DGT)**.

El dataset contiene información detallada sobre accidentes ocurridos en España, incluyendo:
- Tipo de vehículo implicado
- Gravedad del accidente
- Tipo de vía
- Localización
- Variables temporales (fecha y hora)

## Objetivo
El objetivo de este análisis es **identificar patrones y factores de riesgo** asociados a los accidentes de tráfico, con especial atención a:
- Diferencias entre tipos de vehículos
- Relación entre tipo de vía y gravedad del accidente
- Patrones geográficos y temporales

## Preguntas de análisis
Este EDA busca responder, entre otras, a las siguientes preguntas:
1. ¿Qué tipos de vehículos están más implicados en accidentes con víctimas?
2. ¿Existen diferencias significativas en la gravedad de los accidentes según el tipo de vía?
3. ¿Cómo se distribuyen los accidentes a nivel geográfico (provincias)?
4. ¿Se observan patrones temporales claros (hora del día, día de la semana)?


# Fase Inicial

## Importar librerías

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)

## Cargar Excel

In [3]:
file_path = "../data/raw/accidentes_victimas_2024.xlsx" 
diccionario_excel = "../data/raw/diccionario_codigos_2024.xlsx" 
accidentes = pd.read_excel(file_path)

accidentes.head()

,ID_ACCIDENTE,ANYO,MES,DIA_SEMANA,HORA,COD_PROVINCIA,COD_MUNICIPIO,ISLA,ZONA,ZONA_AGRUPADA,CARRETERA,KM,SENTIDO_1F,TITULARIDAD_VIA,TIPO_VIA,TIPO_ACCIDENTE,TOTAL_MU24H,TOTAL_HG24H,TOTAL_HL24H,TOTAL_VICTIMAS_24H,TOTAL_MU30DF,TOTAL_HG30DF,TOTAL_HL30DF,TOTAL_VICTIMAS_30DF,TOTAL_VEHICULOS,TOT_PEAT_MU24H,TOT_BICI_MU24H,TOT_CICLO_MU24H,TOT_MOTO_MU24H,TOT_TUR_MU24H,TOT_FURG_MU24H,TOT_CAM_MENOS3500_MU24H,TOT_CAM_MAS3500_MU24H,TOT_BUS_MU24H,TOT_OTRO_MU24H,TOT_SINESPECIF_MU24H,TOT_PEAT_MU30DF,TOT_BICI_MU30DF,TOT_CICLO_MU30DF,TOT_MOTO_MU30DF,TOT_TUR_MU30DF,TOT_FURG_MU30DF,TOT_CAM_MENOS3500_MU30DF,TOT_CAM_MAS3500_MU30DF,TOT_BUS_MU30DF,TOT_VMP_MU30DF,TOT_OTRO_MU30DF,TOT_SINESPECIF_MU30DF,NUDO,NUDO_INFO,CARRETERA_CRUCE,PRIORI_NORMA,PRIORI_AGENTE,PRIORI_SEMAFORO,PRIORI_VERT_STOP,PRIORI_VERT_CEDA,PRIORI_HORIZ_STOP,PRIORI_HORIZ_CEDA,PRIORI_MARCAS,PRIORI_PEA_NO_ELEV,PRIORI_PEA_ELEV,PRIORI_MARCA_CICLOS,PRIORI_CIRCUNSTANCIAL,PRIORI_OTRA,CONDICION_NIVEL_CIRCULA,CONDICION_FIRME,CONDICION_ILUMINACION,CONDICION_METEO,CONDICION_NIEBLA,CONDICION_VIENTO,VISIB_RESTRINGIDA_POR,ACERA,TRAZADO_PLANTA
0,1,2024,1,3,22,1,0,NaN,1,1,A-2622,33.0,1,3,6,18.0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,NaN,NaN,999,999,999,999,999,999,999,999,999,999,999,999,999,1,1,6,1,NaN,NaN,1,998,1
1,2,2024,1,6,23,1,1036,NaN,1,1,A-625,369.0,1,3,5,16.0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,NaN,NaN,999,999,999,999,999,999,999,999,999,999,999,999,999,1,3,6,4,NaN,NaN,1,998,3
2,3,2024,1,7,1,1,0,NaN,1,1,AP-1,82.0,1,2,2,12.0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,NaN,NaN,999,999,999,999,999,999,999,999,999,999,999,999,999,1,3,6,3,NaN,NaN,1,998,1
3,4,2024,1,7,8,1,0,NaN,1,1,A-2522,32.0,1,3,6,13.0,0,0,2,2,0,0,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,NaN,NaN,999,999,999,999,999,999,999,999,999,999,999,999,999,1,3,1,3,NaN,NaN,1,998,3
4,5,2024,1,4,17,1,0,NaN,1,1,A-3600,19.0,1,3,14,15.0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,NaN,NaN,999,999,999,999,999,999,999,999,999,999,999,999,999,1,3,1,2,NaN,NaN,1,998,1


## Inspección inicial de los datos

Esta celda muestra:
- Información general del dataframe
- Columnas con valores nulos
- Totales de víctimas por tipo de vehículo

In [4]:
# Definimos los grupos lógicos
accidentes['MUERTOS_PEATON'] = accidentes['TOT_PEAT_MU24H']
accidentes['MUERTOS_BICI'] = accidentes['TOT_BICI_MU24H']

# Unificamos Motos (Ciclomotores + Motocicletas)
accidentes['MUERTOS_MOTO'] = (
    accidentes['TOT_CICLO_MU24H'] + 
    accidentes['TOT_MOTO_MU24H']
)

# Unificamos Camiones (Menos y más de 3500kg + Furgonetas)
accidentes['MUERTOS_TRANSPORTE'] = (
    accidentes['TOT_FURG_MU24H'] + 
    accidentes['TOT_CAM_MENOS3500_MU24H'] + 
    accidentes['TOT_CAM_MAS3500_MU24H']
)

# Turismos, Autobuses y Otros
accidentes['MUERTOS_COCHE'] = accidentes['TOT_TUR_MU24H']
accidentes['MUERTOS_BUS'] = accidentes['TOT_BUS_MU24H']
accidentes['MUERTOS_OTROS'] = (
    accidentes['TOT_OTRO_MU24H'] + 
    accidentes['TOT_SINESPECIF_MU24H']
)

# Creamos la lista de nuestras nuevas columnas para el análisis
cols_unificadas = [
    'MUERTOS_COCHE', 'MUERTOS_MOTO', 'MUERTOS_PEATON', 
    'MUERTOS_TRANSPORTE', 'MUERTOS_BICI', 'MUERTOS_BUS', 'MUERTOS_OTROS'
]


# Limpieza

## Limpieza de columnas y valores 

Eliminar estas columnas de análisis inicial ya que tienen más 90% nulos y aportan poco valor al principio


In [5]:
# Eliminar columnas con más de 90% nulos
threshold = 0.9
accidentes_limpios = accidentes.drop(columns=[col for col in accidentes.columns if accidentes[col].isnull().mean() > threshold])

In [6]:
# Lista de tus columnas de vehículos (las originales y las unificadas)
columnas_conteo = [
    "TOT_PEAT_MU24H", "TOT_BICI_MU24H", "TOT_CICLO_MU24H", "TOT_MOTO_MU24H",
    "TOT_TUR_MU24H", "TOT_FURG_MU24H", "TOT_CAM_MENOS3500_MU24H",
    "TOT_CAM_MAS3500_MU24H", "TOT_BUS_MU24H", "TOT_OTRO_MU24H", "TOT_SINESPECIF_MU24H",
    "MUERTOS_MOTO", "MUERTOS_COCHE", "MUERTOS_TRANSPORTE", "MUERTOS_PEATON" # Tus nuevas columnas
]

# Rellenamos con 0 los nulos en estas columnas para poder sumar
accidentes_limpios[columnas_conteo] = accidentes_limpios[columnas_conteo].fillna(0)

## Crear nueva columna derivada 
Diferencia entre el total de [MUERTOS_VEHICULOS] y [DIF_MUERTOS]

In [7]:
# Sumamos nuestras nuevas categorías unificadas
accidentes_limpios["MUERTOS_VEHICULOS_SUMA"] = accidentes_limpios[cols_unificadas].sum(axis=1)

# Calculamos la diferencia con el total oficial
accidentes_limpios["DIF_MUERTOS"] = accidentes_limpios["TOTAL_MU24H"] - accidentes_limpios["MUERTOS_VEHICULOS_SUMA"]

# Verificamos si hay errores
print("Estadísticas de la diferencia (Debe ser cercano a 0):")
print(accidentes_limpios["DIF_MUERTOS"].describe())

# Bonus: Ver si hay algún descuadre grave
descuadres = accidentes_limpios[accidentes_limpios["DIF_MUERTOS"] != 0]
if not descuadres.empty:
    print(f"⚠️ Atención: Hay {len(descuadres)} filas donde la suma no cuadra.")

Estadísticas de la diferencia (Debe ser cercano a 0):
count    101996.000000
mean          0.000078
std           0.008856
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: DIF_MUERTOS, dtype: float64
⚠️ Atención: Hay 8 filas donde la suma no cuadra.


In [8]:
# Ajustamos los errores: si hay diferencia, la sumamos a 'OTROS'
accidentes_limpios['MUERTOS_OTROS'] = accidentes_limpios['MUERTOS_OTROS'] + accidentes_limpios['DIF_MUERTOS']

# Recalculamos la validación para confirmar el 0 absoluto
accidentes_limpios["MUERTOS_VEHICULOS_SUMA"] = accidentes_limpios[cols_unificadas].sum(axis=1)
accidentes_limpios["DIF_MUERTOS"] = accidentes_limpios["TOTAL_MU24H"] - accidentes_limpios["MUERTOS_VEHICULOS_SUMA"]

print(f"Errores restantes: {len(accidentes_limpios[accidentes_limpios['DIF_MUERTOS'] != 0])}")

Errores restantes: 0


In [9]:

# Cargamos el archivo de Excel (solo los metadatos)
xls = pd.ExcelFile('../data/raw/diccionario_codigos_2024.xlsx')

# Listamos todos los nombres de las pestañas
nombres_pestanas = xls.sheet_names

print(f"El diccionario tiene {len(nombres_pestanas)} pestañas:")
print("-" * 30)
for nombre in nombres_pestanas:
    print(f"• {nombre}")

El diccionario tiene 38 pestañas:
------------------------------
• DIA_SEMANA
• COD_PROVINCIA
• COD_MUNICIPIO
• ISLA
• ZONA
• ZONA_AGRUPADA
• CARRETERA
• KM
• SENTIDO_1F
• TITULARIDAD_VIA
• TIPO_VIA
• TIPO_ACCIDENTE
• TOTALIZADORES
• NUDO
• NUDO_INFO
• CARRETERA_CRUCE
• PRIORI_NORMA
• PRIORI_AGENTE
• PRIORI_SEMAFORO
• PRIORI_VERT_STOP
• PRIORI_VERT_CEDA
• PRIORI_HORIZ_STOP
• PRIORI_HORIZ_CEDA
• PRIORI_MARCAS
• PRIORI_PEA_NO_ELEV
• PRIORI_PEA_ELEV
• PRIORI_MARCA_CICLOS
• PRIORI_CIRCUNSTANCIAL
• PRIORI_OTRA
• CONDICION_NIVEL_CIRCULA
• CONDICION_FIRME
• CONDICION_ILUMINACION
• CONDICION_METEO
• CONDICION_NIEBLA
• CONDICION_VIENTO
• VISIB_RESTRINGIDA_POR
• ACERA
• TRAZADO_PLANTA


## Unificar diccionario

In [21]:
# 1. Ajustamos el mapeo con los nombres reales que te han salido
columnas_a_mapear = {
    'COD_PROVINCIA': 'COD_PROVINCIA',
    'TIPO_VIA': 'TIPO_VIA',
    'DIA_SEMANA': 'DIA_SEMANA',
    'TIPO_ACCIDENTE': 'TIPO_ACCIDENTE',
    'ZONA_AGRUPADA': 'ZONA_AGRUPADA',
    'TRAZADO_PLANTA': 'TRAZADO_PLANTA',
    'NUDO': 'NUDO',
    'CONDICION_METEO': 'CONDICION_METEO', 
    'CONDICION_ILUMINACION': 'CONDICION_ILUMINACION',
    'CONDICION_FIRME': 'CONDICION_FIRME',
    'CONDICION_VIENTO': 'CONDICION_VIENTO',
    'VISIB_RESTRINGIDA_POR': 'VISIB_RESTRINGIDA_POR'
}

# 2. Volvemos a ejecutar el bucle (ahora no saltará ninguna)
with pd.ExcelFile(diccionario_excel) as xls:
    for col_csv, pestana in columnas_a_mapear.items():
        if col_csv in accidentes_limpios.columns:
            df_dict = pd.read_excel(xls, sheet_name=pestana, header=1)
            df_dict.columns = df_dict.columns.str.strip()
            mapa = dict(zip(df_dict["Valor"], df_dict["Etiqueta"].astype(str).str.strip()))
            
            accidentes_limpios[f"{col_csv}_NOMBRE"] = accidentes_limpios[col_csv].map(mapa)
            print(f"✅  Mapeada: {col_csv}")

✅  Mapeada: COD_PROVINCIA
✅  Mapeada: TIPO_VIA
✅  Mapeada: DIA_SEMANA
✅  Mapeada: TIPO_ACCIDENTE
✅  Mapeada: ZONA_AGRUPADA
✅  Mapeada: TRAZADO_PLANTA
✅  Mapeada: NUDO
✅  Mapeada: CONDICION_METEO
✅  Mapeada: CONDICION_ILUMINACION
✅  Mapeada: CONDICION_FIRME
✅  Mapeada: VISIB_RESTRINGIDA_POR


# Análisis básico

### TOP 5 ACCIDENTES MÁS MORTALES

In [11]:
# Media de muertos por cada accidente según el tipo
letalidad_motivo = accidentes_limpios.groupby('TIPO_ACCIDENTE_NOMBRE')['TOTAL_MU24H'].mean().sort_values(ascending=False)
print("Top 5 Accidentes más Mortales:\n", letalidad_motivo.head(5))

Top 5 Accidentes más Mortales:
 TIPO_ACCIDENTE_NOMBRE
Salida de la vía por la derecha con despeñamiento      0.113095
Salida de la vía por la izquierda con despeñamiento    0.100457
Frontal                                                0.080805
Salida de la vía por la izquierda con colisión         0.045486
Salida de la vía por la derecha con colisión           0.043200
Name: TOTAL_MU24H, dtype: float64


### TOTAL DE FALLECIDOS POR CLIMA

In [12]:
# Suma total de muertos por clima
muertos_clima = accidentes_limpios.groupby('CONDICION_METEO_NOMBRE')['TOTAL_MU24H'].sum().sort_values(ascending=False)
print("\nMuertos según el Clima:\n", muertos_clima)


Muertos según el Clima:
 CONDICION_METEO_NOMBRE
Despejado          1285
Nublado             126
Lluvia débil         59
Lluvia fuerte        30
Se desconoce         19
Granizando            1
Nevando               1
Sin especificar       1
Name: TOTAL_MU24H, dtype: int64


### MOTORISTAS FALLECIDOS POR ZONA

In [13]:
# Comparativa de muertos en moto según la zona
moto_zona = accidentes_limpios.groupby('ZONA_AGRUPADA_NOMBRE')['MUERTOS_MOTO'].sum()
print("\nFallecidos en Moto por Zona:\n", moto_zona)


Fallecidos en Moto por Zona:
 ZONA_AGRUPADA_NOMBRE
VÍAS INTERURBANAS    299
VÍAS URBANAS         124
Name: MUERTOS_MOTO, dtype: int64


In [14]:
# Buscamos cualquier columna que contenga la palabra PRIOR
cols_reales_csv = [col for col in accidentes_limpios.columns if 'PRIOR' in col]
print("Nombres reales en tu CSV:", cols_reales_csv)

Nombres reales en tu CSV: ['PRIORI_NORMA', 'PRIORI_AGENTE', 'PRIORI_SEMAFORO', 'PRIORI_VERT_STOP', 'PRIORI_VERT_CEDA', 'PRIORI_HORIZ_STOP', 'PRIORI_HORIZ_CEDA', 'PRIORI_MARCAS', 'PRIORI_PEA_NO_ELEV', 'PRIORI_PEA_ELEV', 'PRIORI_MARCA_CICLOS', 'PRIORI_CIRCUNSTANCIAL', 'PRIORI_OTRA']


# Exportar datos


In [15]:
# Lista de todas tus columnas de prioridad reales
cols_prioridad_reales = [
    'PRIORI_NORMA', 'PRIORI_AGENTE', 'PRIORI_SEMAFORO', 'PRIORI_VERT_STOP', 
    'PRIORI_VERT_CEDA', 'PRIORI_HORIZ_STOP', 'PRIORI_HORIZ_CEDA', 'PRIORI_MARCAS', 
    'PRIORI_PEA_NO_ELEV', 'PRIORI_PEA_ELEV', 'PRIORI_MARCA_CICLOS', 
    'PRIORI_CIRCUNSTANCIAL', 'PRIORI_OTRA'
]

with pd.ExcelFile(diccionario_excel) as xls:
    for col in cols_prioridad_reales:
        if col in accidentes_limpios.columns:
            try:
                # Intentamos cargar la pestaña (usamos el mismo nombre de la columna)
                df_dict = pd.read_excel(xls, sheet_name=col, header=1)
                df_dict.columns = df_dict.columns.str.strip()
                
                # Creamos el mapa
                mapa = dict(zip(df_dict["Valor"], df_dict["Etiqueta"].astype(str).str.strip()))
                
                # Creamos la columna mapeada
                accidentes_limpios[f"{col}_NOMBRE"] = accidentes_limpios[col].map(mapa)
                print(f"✅ Columna creada: {col}_NOMBRE")
            except Exception as e:
                print(f"⚠️ No se pudo mapear {col}: {e}")

✅ Columna creada: PRIORI_NORMA_NOMBRE
✅ Columna creada: PRIORI_AGENTE_NOMBRE
✅ Columna creada: PRIORI_SEMAFORO_NOMBRE
✅ Columna creada: PRIORI_VERT_STOP_NOMBRE
✅ Columna creada: PRIORI_VERT_CEDA_NOMBRE
✅ Columna creada: PRIORI_HORIZ_STOP_NOMBRE
✅ Columna creada: PRIORI_HORIZ_CEDA_NOMBRE
✅ Columna creada: PRIORI_MARCAS_NOMBRE
✅ Columna creada: PRIORI_PEA_NO_ELEV_NOMBRE
✅ Columna creada: PRIORI_PEA_ELEV_NOMBRE
✅ Columna creada: PRIORI_MARCA_CICLOS_NOMBRE
✅ Columna creada: PRIORI_CIRCUNSTANCIAL_NOMBRE
✅ Columna creada: PRIORI_OTRA_NOMBRE


In [23]:
# 1. Calculamos el Índice de Letalidad (Asegúrate de que esta celda se ejecute)
accidentes_limpios['INDICE_LETALIDAD'] = (
    (accidentes_limpios['TOTAL_MU24H'] / accidentes_limpios['TOTAL_VICTIMAS_24H']) * 100
).fillna(0)

# 2. Verificamos los nombres reales de Carretera y KM
# En los datos de la DGT suelen ser 'CARRETERA' y 'KM' o 'PUNTO_KILOMETRICO'
# Este print te ayudará a confirmar si existen
print("Columnas actuales:", accidentes_limpios.columns.tolist())

# 3. Lista base actualizada (Ajustada para evitar KeyErrors)
columnas_base = [
    'ID_ACCIDENTE', 'HORA', 'MES', 'TOTAL_MU24H', 'TOTAL_HG24H', 'TOTAL_HL24H', 
    'TOTAL_VICTIMAS_24H', 'INDICE_LETALIDAD', 'TOTAL_VEHICULOS',
    'MUERTOS_COCHE', 'MUERTOS_MOTO', 'MUERTOS_PEATON', 'MUERTOS_TRANSPORTE', 'MUERTOS_BICI', 'VISIB_RESTRINGIDA_POR'
]

# Añadimos las geográficas solo si existen en el DataFrame original
cols_geo = ['CARRETERA', 'KM', 'SENTIDO_1F']
for col in cols_geo:
    if col in accidentes_limpios.columns:
        columnas_base.append(col)
    else:
        print(f"⚠️ Aviso: La columna {col} no se encontró en el dataset original.")

# 4. Detectamos dinámicamente las mapeadas (_NOMBRE)
todas_las_mapeadas = [col for col in accidentes_limpios.columns if col.endswith('_NOMBRE')]

# 5. Combinamos sin duplicados
columnas_finales = list(dict.fromkeys(columnas_base + todas_las_mapeadas))

# 6. FILTRO DE SEGURIDAD FINAL: Solo columnas que REALMENTE existen
columnas_finales_reales = [c for c in columnas_finales if c in accidentes_limpios.columns]

# 7. Exportación
ruta_maestro = "../data/processed/viarisk_master_2024.csv"
accidentes_limpios[columnas_finales_reales].to_csv(ruta_maestro, index=False, sep=';', encoding='utf-8')

print(f"🚀 ¡ÉXITO! Guardado con {len(columnas_finales_reales)} columnas.")

Columnas actuales: ['ID_ACCIDENTE', 'ANYO', 'MES', 'DIA_SEMANA', 'HORA', 'COD_PROVINCIA', 'COD_MUNICIPIO', 'ZONA', 'ZONA_AGRUPADA', 'CARRETERA', 'KM', 'SENTIDO_1F', 'TITULARIDAD_VIA', 'TIPO_VIA', 'TIPO_ACCIDENTE', 'TOTAL_MU24H', 'TOTAL_HG24H', 'TOTAL_HL24H', 'TOTAL_VICTIMAS_24H', 'TOTAL_MU30DF', 'TOTAL_HG30DF', 'TOTAL_HL30DF', 'TOTAL_VICTIMAS_30DF', 'TOTAL_VEHICULOS', 'TOT_PEAT_MU24H', 'TOT_BICI_MU24H', 'TOT_CICLO_MU24H', 'TOT_MOTO_MU24H', 'TOT_TUR_MU24H', 'TOT_FURG_MU24H', 'TOT_CAM_MENOS3500_MU24H', 'TOT_CAM_MAS3500_MU24H', 'TOT_BUS_MU24H', 'TOT_OTRO_MU24H', 'TOT_SINESPECIF_MU24H', 'TOT_PEAT_MU30DF', 'TOT_BICI_MU30DF', 'TOT_CICLO_MU30DF', 'TOT_MOTO_MU30DF', 'TOT_TUR_MU30DF', 'TOT_FURG_MU30DF', 'TOT_CAM_MENOS3500_MU30DF', 'TOT_CAM_MAS3500_MU30DF', 'TOT_BUS_MU30DF', 'TOT_VMP_MU30DF', 'TOT_OTRO_MU30DF', 'TOT_SINESPECIF_MU30DF', 'NUDO', 'NUDO_INFO', 'PRIORI_NORMA', 'PRIORI_AGENTE', 'PRIORI_SEMAFORO', 'PRIORI_VERT_STOP', 'PRIORI_VERT_CEDA', 'PRIORI_HORIZ_STOP', 'PRIORI_HORIZ_CEDA', 'PRIORI